In [ ]:
!git clone https://github.com/EmanueleCeglia/PDF-Metadata-Extractor.git

In [2]:
%cd PDF-Metadata-Extractor

/content/PDF-Metadata-Extractor


In [ ]:
!pip install -r requirements.txt

In [ ]:
!apt-get install ghostscript -y

## Extract Dates

In [4]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from pdf_dates_finder import PDFDatesFinderSemanticSearch  #old method
from pdf_dates_finder import PDFDatesFinderSpace   #new method
import os
import re
from tqdm import tqdm

Download model

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("Open-Orca/OpenOrca-Platypus2-13B")
model = AutoModelForCausalLM.from_pretrained("Open-Orca/OpenOrca-Platypus2-13B", device_map="auto", load_in_8bit=True)

In [6]:
def post_process_response_dates(text):
    start_date_pattern = r"Start date:\s*([0-9]+[a-z]*\s+[A-Z][a-z]+\s+[0-9]{4})"
    end_date_pattern = r"End date:\s*([0-9]+[a-z]*\s+[A-Z][a-z]+\s+[0-9]{4})"

    start_date_match = re.search(start_date_pattern, text)
    end_date_match = re.search(end_date_pattern, text)

    result = {}
    if start_date_match and end_date_match:
        result['Start date'] = start_date_match.group(1)
        result['End date'] = end_date_match.group(1)
    else:
        return None

    return result

# second level filter
def find_dates_regex(string):
    pattern = r'\b\d{1,2}/\d{1,2}/\d{4}\b|\b\d{1,2} [A-Za-z]+ \d{4}\b|\b\d{1,2}(?:st|nd|rd|th)? [A-Za-z]+ \d{4}\b|\b[A-Za-z]+ \d{1,2}(?:st|nd|rd|th)? \d{4}\b'
    dates = re.findall(pattern, string)
    if len(dates) >= 2:
        return f"Start date: {dates[0]} End date: {dates[1]}"
    elif len(dates) == 1:
        return f"Start date: {dates[0]} End date: None"
    else:
        return "No dates found"

Dates extraction with OLD METHOD

In [ ]:
insurances_folder_path = "/content/PDF-Metadata-Extractor/Insurances"

extracted_dates = {}

for root, dirs, files in os.walk(insurances_folder_path):
    for file_name in tqdm(files):
        full_file_path = os.path.join(root, file_name)

        extraction = PDFDatesFinderSemanticSearch(full_file_path)
        extraction.load_pdf()
        extraction.process_text()
        dates = extraction.find_dates()

        for value in dates.values():
          for phrase in value.values():
            if len(phrase):
              sentence = phrase[0]

              prompt = "Find start date and end date from the following sentence: " + sentence
              inputs = tokenizer(prompt, return_tensors="pt")
              generate_ids = model.generate(inputs.input_ids, max_length=1000)
              response = tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
              response = post_process_response_dates(response)

              if response:
                extracted_dates[str(file_name)] = response
                print(str(file_name) + ' EXTRACTED!')
              else:
                print(str(file_name) + ' NOT FOUND!')

Dates extraction with NEW METHOD

In [7]:
insurances_folder_path = "/content/PDF-Metadata-Extractor/Insurances"

extracted_dates = {}

for root, dirs, files in os.walk(insurances_folder_path):
    for file_name in tqdm(files):
        full_file_path = os.path.join(root, file_name)

        extractor = PDFDatesFinderSpace(full_file_path)
        pages, tables = extractor.extract_mytext()
        paragraphs = [extractor.identify_paragraphs_space(page) for page in pages]
        #print(paragraphs)

        # keywords filter 1th level
        check_kw = False
        for phrase in paragraphs:
          for sentence in phrase:
            if re.search(r'\bperiod\b', sentence, re.IGNORECASE):
              check_kw = True

        if check_kw:
          # Use a list comprehension with regex to keep only the phrases that contain the word "period" (case-insensitive)
          paragraphs = [sublist for sublist in paragraphs if any(re.search(r'\bperiod\b', phrase, re.IGNORECASE) for phrase in sublist)]

          # Now, further filter each sublist to keep only the phrases that contain the word "period"
          paragraphs = [[phrase for phrase in sublist if re.search(r'\bperiod\b', phrase, re.IGNORECASE)] for sublist in paragraphs]


        responses = []
        for phrase in paragraphs:
          if len(phrase):
            for sentence in phrase:
              sentcence = sentence.replace(',', '')
              prompt = "Find start date and end date from the following sentence: " + sentence
              inputs = tokenizer(prompt, return_tensors="pt")
              generate_ids = model.generate(inputs.input_ids, max_length=200)
              response = tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
              response = post_process_response_dates(response)

              if response:
                  responses.append(response)
                  #print(response)

        if len(responses)==0:
          # Parse with regex for prompt fails
          for phrase in paragraphs:
            if len(phrase):
              if len(phrase)>1:
                phrase = [' '.join(phrase)]
              for sentence in phrase:
                  sentence = sentence.replace(',','')
                  response = find_dates_regex(sentence)
                  responses.append(response)

        if responses:
          print(str(file_name) + ' EXTRACTED!')
          extracted_dates[str(file_name)] = responses
        else:
          print(str(file_name) + ' NOT FOUND!')

  0%|          | 0/31 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1468: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(
  3%|▎         | 1/31 [01:02<31:06, 62.21s/it]

policy_10.pdf EXTRACTED!


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1468: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(
  6%|▋         | 2/31 [02:57<45:17, 93.69s/it]

policy_5.pdf EXTRACTED!


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1468: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(
 10%|▉         | 3/31 [04:34<44:23, 95.14s/it]

...

policy_63.pdf EXTRACTED!


 97%|█████████▋| 30/31 [49:30<01:35, 95.62s/it] 

policy_22.pdf NOT FOUND!


Input length of input_ids is 300, but `max_length` is set to 200. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1468: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(
100%|██████████| 31/31 [50:17<00:00, 97.35s/it]

policy_39.pdf EXTRACTED!


In [8]:
extracted_dates

{'policy_10.pdf': [{'Start date': '1st January 2022',
   'End date': '31st December 2022'}],
 'policy_5.pdf': [{'Start date': '01 November 2021',
   'End date': '31 October 2022'},
  {'Start date': '01 November 2021', 'End date': '31 October 2022'}],
 'policy_2.pdf': ['Start date: 01 January 2021 End date: 31 December 2020'],
 'policy_97.pdf': [{'Start date': '01 September 2021',
   'End date': '31 August 2022'},
  {'Start date': '01 September 2021', 'End date': '31 August 2022'}],
 'policy_67.pdf': ['Start date: 31st October 2022 End date: 31st October 2023'],
 'policy_26.pdf': [{'Start date': '1st November 2022',
   'End date': '31st October 2023'}],
 'policy_40.pdf': [{'Start date': '6th March 2022',
   'End date': '6th March 2023'}],
 'policy_52.pdf': [{'Start date': '31st May 2022',
   'End date': '30th May 2023'}],
 'policy_100.pdf': [{'Start date': '1st January 2021',
   'End date': '1st January 2022'},
  {'Start date': '1st January 2021', 'End date': '1st January 2022'}],
 'pol